<a href="https://colab.research.google.com/github/zyberg2091/spooky-author-identification/blob/master/Spooky_Author_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input,Dense,LSTM,SimpleRNN,Dropout,Conv1D,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import Regularizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import pandas as pd
df=pd.read_csv("train.csv")
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
df.columns.values

array(['id', 'text', 'author'], dtype=object)

In [ ]:
df=pd.get_dummies(data=df,columns=['author'])

In [ ]:
df.head()

,id,text,author_EAP,author_HPL,author_MWS
0,id26305,"This process, however, afforded me no means of...",1,0,0
1,id17569,It never once occurred to me that the fumbling...,0,1,0
2,id11008,"In his left hand was a gold snuff box, from wh...",1,0,0
3,id27763,How lovely is spring As we looked from Windsor...,0,0,1
4,id12958,"Finding nothing else, not even gold, the Super...",0,1,0


In [ ]:
#df['author']=df['author'].map({'EAP':0,'HPL':1,'MWS':2})

In [ ]:
x=df.iloc[:,:2].values
y=df.iloc[:,[-3,-2,-1]].values

In [ ]:
x.shape,y.shape

((19579, 2), (19579, 3))

In [ ]:
tokenizer=Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(x[:,1])
vocab=len(tokenizer.word_index)
sequence=tokenizer.texts_to_sequences(x[:,1])
x_pad=pad_sequences(sequence)
print(vocab)

25944


In [ ]:
x_pad.shape

(19579, 861)

In [ ]:
x_pad[1].shape

(861,)

In [ ]:
#architecture
T=x_pad.shape[1]

i=Input(shape=(T,))
x=Embedding(vocab+1,64)(i)
x=LSTM(128)(x)
x=Dense(64,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(3,activation='softmax')(x)

model=Model(i,x)

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 861)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 861, 64)           1660480   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 1,767,747
Trainable params: 1,767,747
Non-trainable params: 0
___________________________________________________

In [ ]:
print(T)

861


In [ ]:
x_pad.shape,y.shape

((19579, 861), (19579, 3))

In [ ]:
r=model.fit(x_pad,y,epochs=10,verbose=1)

In [ ]:
plt.plot(r.history['loss'],label='loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'],label='accuracy')
plt.legend()

In [ ]:
test_sentence=["Still, as I urged our leaving Ireland with such inquietude and impatience, my father thought it best to yield."]

tokenizer.fit_on_texts(test_sentence)
test_seq=tokenizer.texts_to_sequences(test_sentence)

In [ ]:
test_pad=pad_sequences(test_seq,maxlen=T,padding='pre')

In [ ]:
test_pad.shape

(1, 861)

In [ ]:
test_pad[0].shape

In [ ]:
predictions=model.predict(test_pad)

In [ ]:
predictions[0]